In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO
import glob
import json
import re

In [2]:
# reading in a single pdf file and parsing the text from the file

def pdf_to_text(pdf_path): 
    
    # PDFminer standard steps
    resource_mgr = PDFResourceManager()
    retstr = BytesIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(resource_mgr, retstr, codec=codec, laparams=laparams)
    interpreter = PDFPageInterpreter(resource_mgr, device)

    # extract text from multi-page pdf
    fp = open(pdf_path, 'rb')
    maxpages = 0
    caching = True
    pagenos = set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, caching=caching, check_extractable=True):
        interpreter.process_page(page)

    result_str = retstr.getvalue()
    
    # clean up
    fp.close()
    device.close()
    retstr.close()
    
    return result_str

In [14]:
# a list of all file paths
fp_list = glob.glob('/Users/liyannie/Dropbox/Westlaw_cases/*.pdf')
case_dict = {}
OUTPUT_FILE = 'amy_cases.json'

for fp in fp_list:
    key = fp.replace('/Users/liyannie/Dropbox/westlaw_revised/', '')
    key = key.replace('.pdf', '')
    key = re.sub("\d", '', key)
    key = key.replace('-', '')
    key = key.lstrip()
    value = pdf_to_text(fp)
    case_dict.update({key:value})


In [15]:
len(case_dict.keys())

854

In [16]:
json.dump(case_dict, open(OUTPUT_FILE, 'w'))